In [1]:
from llama2 import *
from typing import List, Literal, Optional, Tuple, TypedDict
import pandas as pd
import numpy as np
import datasets
import string
from evaluate import evaluator
from evaluate import load

from torch.utils.data import Dataset

/home/chris/Desktop/repos/llama2/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_size='int4'
max_samples=-1

In [3]:
# Parameters
model_size = "int4"
max_samples = -1


## 1 - Load model

In [4]:
#papermill_description=LOADING_MODEL
model_name = "meta-llama/Llama-2-7b-chat-hf"

model = LlamaModel(
    model_name=model_name,
    model_resolution=model_size
)

model.model.to = lambda x: x # Disable device copying

Loading checkpoint shards:   0%|                                                        | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:  50%|████████████████████████                        | 1/2 [00:01<00:01,  1.07s/it]

Loading checkpoint shards: 100%|████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.47it/s]

Loading checkpoint shards: 100%|████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.36it/s]

## 2 - Load dataset

In [5]:
#papermill_description=LOADING_DATA
dataset = datasets.load_dataset('glue', 'mnli', split='validation_matched')

In [6]:
dataset

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 9815
})

In [7]:
dataset[5]

{'premise': "well that would be a help i wish they would do that here we have got so little landfill space left that we're going to run out before the end of this decade and it's really going to be",
 'hypothesis': 'We have plenty of space in the landfill.',
 'label': 2,
 'idx': 5}

## 3 - Define data prep and model inference functions

In [8]:
def format_question(sample: dict) -> str:
    """Format a sample from the squad V2 dataset to question answer string."""
    pretext = (f'{model.B_INST} You are performing natural language inference tasks. '
               'Given a premise and hypothesis, decide whether we have an entailment, neutral or contradiction relation. '
               'Only respond with the words "The response is " and one of "neutral, contradiction, entailment"'
               f' {model.E_INST}\n')  # Llama system directive
    q_a = (f'Premise: {sample["premise"]}\n'
         f'Hypothesis: {sample["hypothesis"]}\n'
         f'Answer: \n\nThe response is:')
    
    return pretext + q_a


def glue_inference(df: pd.DataFrame, model) -> pd.DataFrame:
    """Predict the output extracts for all samples in the input squad format dataset"""
    df_val = df.copy(deep=True)
    df['prediction'] = -1
    df['prob'] = -1
    
    labels = ['neutral', 'entailment', 'contradiction']
    label_ids = [model.tokenizer.encode(label, add_special_tokens=False)[0] for label in labels]

    for idx in range(len(df)):
        with torch.no_grad():
            x = format_question(df.iloc[idx])
            tokens = model.tokenize(x)
            logits = model.model(tokens).logits
            probs = torch.softmax(logits[:,-1,label_ids], dim=1)
            
        df_val.loc[idx, 'prediction'] = probs.argmax().item()
        df_val.loc[idx, 'prob'] = probs.max().item()
        
    df_val.prediction = df_val.prediction.astype(int)
    return df_val

In [9]:
#papermill_description=RUNNING_INFERENCE
pd_dataset = dataset.to_pandas()
if max_samples > 0 and max_samples < len(pd_dataset):
    pd_dataset = pd_dataset.iloc[:max_samples - 1]
df2 = glue_inference(pd_dataset, model)

## 5 - Evaluate performance

In [10]:
#papermill_description=EVALUATION
glue_metric = load("glue", "mnli_matched")

for precision in np.linspace(0,1,10, endpoint=False):
    df_filt = df2[df2.prob > precision]
    
    predictions = df_filt['prediction'].to_list()
    answers = df_filt['label'].to_list()

    results = glue_metric.compute(predictions=predictions, references=answers)
    print(f'Class prob > {precision:.1f}: {results}')

Class prob > 0.0: {'accuracy': 0.3369332654100866}
Class prob > 0.1: {'accuracy': 0.3369332654100866}
Class prob > 0.2: {'accuracy': 0.3369332654100866}
Class prob > 0.3: {'accuracy': 0.3369332654100866}
Class prob > 0.4: {'accuracy': 0.3373863288035149}
Class prob > 0.5: {'accuracy': 0.3395170483987978}
Class prob > 0.6: {'accuracy': 0.34331698998081434}
Class prob > 0.7: {'accuracy': 0.34748010610079577}
Class prob > 0.8: {'accuracy': 0.35404742436631237}
Class prob > 0.9: {'accuracy': 0.36727703859834016}
